In [2]:
import pandas as pd
import numpy as np

import pickle

In [22]:
with open('./preds_0.bin', 'rb') as f:
    preds_df = pickle.load(
        file=f
    )

In [23]:
preds_df.head()

,generated_text,subject,pred,true,corr,category
0,B,abstract_algebra,B,B,1,STEM
1,A,abstract_algebra,A,C,0,STEM
2,D,abstract_algebra,D,D,1,STEM
3,D,abstract_algebra,D,B,0,STEM
4,B,abstract_algebra,B,B,1,STEM


In [24]:
preds_df[['subject', 'corr']].groupby(['subject']).mean()

,corr
subject,
abstract_algebra,0.375


In [25]:
preds_df[['category', 'corr']].groupby(['category']).mean()

,corr
category,
STEM,0.375


In [26]:
total_accuracy = preds_df['corr'].mean()
total_correctness = (preds_df['pred'] != 'I').mean()

print(f"{total_accuracy=}")
print(f"{total_correctness=}")

total_accuracy=0.375
total_correctness=1.0
